In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Edge detector
Apply Sobel filter and non maximum suppression

In [ ]:
def applySobelFilter(img, ksize):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    img_gaussian = cv2.GaussianBlur(gray,(3,3),0)
        
    height = img.shape[0]   #.shape[0] outputs height 
    width = img.shape[1]    #.shape[1] outputs width, shape[2] outputs color channels of image
    matrix_R = np.zeros((height,width))
    
    #   Step 1 - Calculate the image derivatives in x and y (Ix, Iy)
    Ix = cv2.Sobel(src=img_gaussian, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    Iy = cv2.Sobel(src=img_gaussian, ddepth=cv2.CV_64F, dx=0, dy= 1, ksize=5)

#### Non-max suppression
For each pixel, we check if it is a local maximum in its neighborhood in the direction of gradient or not.<br>
If it is a local maximum it is retained as an edge pixel, otherwise suppressed.<br>
For each pixel, the neighboring pixels are located in horizontal, vertical, diagonal directions (0, 45, 90, 135 degree)<br>

Based on these inputs the <i><strong>non-max-suppression steps</strong></i> are:
- Create a matrix initialized to 0 of the same size of the original gradient intensity matrix.
- Identify the edge direction based on the angle value from the angle matrix.
- Check if the pixel in the same direction has a higher intensity than the pixel that is currently processed.
- Return the image processed with the non-max suppression algorithm.<br><br><br>

With the gradient of image:
\begin{equation*}
G = \lbrack G_x, G_y \rbrack
\end{equation*}
Then: 
* mag: gradient magnitude image
\begin{equation*}
\vert G \vert = \sqrt{(G_x)^2 + (G_y)^2}
\end{equation*}
* theta: edge direction
\begin{equation*}
\theta = \arctan(\frac{G_y}{G_x})
\end{equation*}

In [ ]:
def nonMaxSuppression(gray_img):

    img_gaussian = cv2.GaussianBlur(gray_img,(3,3),0)

    Ix = cv2.Sobel(src=img_gaussian, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=3)
    Iy = cv2.Sobel(src=img_gaussian, ddepth=cv2.CV_64F, dx=0, dy= 1, ksize=3)
    
    mag = np.hypot(Ix, Iy)
    mag = mag / mag.max() * 255
    theta = np.arctan2(Iy, Ix)

    height = mag.shape[0]
    width = mag.shape[1]
    non_max = np.zeros((height, width), dtype=np.int32)
    
    # radian to degree
    angle = theta * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = mag[i, j+1]
                    r = mag[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = mag[i+1, j-1]
                    r = mag[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = mag[i+1, j]
                    r = mag[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = mag[i-1, j-1]
                    r = mag[i+1, j+1]

                if (mag[i,j] >= q) and (mag[i,j] >= r):
                    non_max[i,j] = mag[i,j]
                else:
                    non_max[i,j] = 0

            except IndexError as e:
                pass
    
    return non_max